In [225]:
import tensorflow
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [226]:
#Load the trained, scaler pickle, ohe
model = load_model("model.h5")

#Load the encoder and scaler
with open("onehot_encoder.pkl", "rb") as file:
    onehot_encoder_geo = pickle.load(file)

with open("label_encoder_gender.pkl", "rb") as file:
    lable_encoder_gender = pickle.load(file)

with open("scaler.pkl", "rb") as file:
    scaler = pickle.load(file)


In [227]:
['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited']

['CreditScore',
 'Geography',
 'Gender',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Exited']

In [228]:
#Example Input data
input_data = {
    "CreditScore" : 619, 
    'Geography' : "France", 
    "Gender":"Female",
    "Age":42,
    'Tenure':2,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':10134
}

In [229]:
#One hot encode "Geopraphy"
geo_encoded = onehot_encoder_geo.transform([[input_data["Geography"]]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df

d:\Learning\ML\042_end_to_end_project_using_ANN\Implementation\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [230]:
input_data_df = pd.DataFrame([input_data])

In [231]:
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,60000,2,1,1,10134


In [232]:
#Label Encoding Gender
input_data_df["Gender"] = lable_encoder_gender.transform(input_data_df["Gender"])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,60000,2,1,1,10134


In [233]:
#Combining ohe with input data
input_data_df = pd.concat([input_data_df.drop("Geography",axis=1), geo_encoded_df], axis=1)
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,60000,2,1,1,10134,1.0,0.0,0.0


In [234]:
#Scaling the Input data
input_scaled = scaler.transform(input_data_df)

In [235]:
input_scaled

array([[-0.33880827, -1.09499335,  0.29493847, -1.04241787, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -1.56996921,  1.00150113,
        -0.57946723, -0.57638802]])

In [236]:
#Predict Churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step


array([[0.12220188]], dtype=float32)

In [237]:
prediction_prob = prediction[0][0]

In [238]:
print(prediction_prob)

0.12220188


In [239]:
if prediction_prob > 0.5 :
    print("The custmer is likely to churn.")
else:
    print("The custmer is not likely to churn.")

The custmer is not likely to churn.
